### Titre:
Identification des zones habitées non éclairées par croisement **GHSL-VIIRS**: cas de la commune de **Bokidiawé**
### Objectif
Identifier les zones habitées mais non éclairées dans la commune de **Bokidiawé** afin d'évaluer la vulnérabilité à l'obscurité nocturne et aider à la planification des infrastructures.
### Données
1. **VIIRS (Visible Infrared Imaging Radiometer Suite)**
   - Dataset: `NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG`. Il s'agit d'une image composite de la radiance moyenne mensuelle générée à l'aide des données nocturne de la bande Day/Night(DNB).
   - Résolution: 450m
   - Bande de fréquence: `avg_rad`, valeur moyenne de la radiance DNB
3. **GHSL (Global Humain Settlement Layer)**
   - Dataset `JRC/GHSL/P2023A/GHS_POP/2020`. Cette ensemble de données représente la distribution spatiale de la population résidentielle, exprimée en nombre absolue d'habitants de la cellule.
   - Résolution: 100m
   - Bande: `population_count`, nombre d'habitants par pixel
5. **Limites administratives**: Polygons délimitant les communes du Sénégal
### Méthodologie
- **filtrage spatial**: Découpage de VIIRS et GHS_POP sur la commune de *Bokidiawé*
- **Reprojection**: Aligner les images sur le même système de coordonnées(CRS), la même résolution et le même point d'origine.
- **Seuil de densité**: Densité officielle de la commune de Bokidiawé en 124.4 hab/km2 convertie en hab/pixel (à 100m).
- **Seuil de lumière VIIRS**: Intensité lumineuse, seuil définie par test visuel
- **Croisement**: `habité et non éclairé` donne zones vulnérables à l'obscurité
### Résultat attendu
- Carte des **zones habitées mais non éclairées**
- Statistiques:
  - % de la superficie des zones vulnérables (en ha)
  - Nombre estimé d'habitants concernés

In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [3]:
ee.Initialize()

#### Charger les données

In [4]:
def get_data(region):

    # Données de la population (WorldPop)
    #ghsl_pop = ee.Image('JRC/GHSL/P2023A/GHS_POP/2020').clip(region)
    ghsl_pop = ee.ImageCollection('JRC/GHSL/P2023A/GHS_POP')\
                    .filterDate('2025-01-01', '2026-01-01')\
                    .first()\
                    .clip(region)
   
   
    """ Données de la lumière nocturne
        Fusiooner plusieurs images en une seule image composite, en prenant par défaut la valeur du pixel la plus haute dans l'ordre de
       la collection."""
    viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
                .mosaic()\
                .clip(region)  
                
                   
    return ghsl_pop, viirs


In [5]:
# Fonction permet charger les limites administratives des communes du Sénégal et de selectionner la commune à étudier
def commune(commune_name):
    df = pd.read_csv('donnees.csv', sep=';', usecols=['nom', 'geometry'])
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    geom = gdf[gdf['nom'] == commune_name]['geometry'].iloc[0]
    
    geom_json = geom.__geo_interface__
    ee_geom = ee.Geometry(geom_json)
    return ee_geom

In [6]:
# Définir une commune d'intérêt (Ex: Bokidiawé)
ee_geom = commune('BOKIDIAWE')
population, viirs = get_data(ee_geom)

#### Reprojection des images
L'alignement des images raster est une étape essentielle dans tout projet d'analyse géospatiale impliquant plusieurs couches(ex. population, eéclairage nocturne). Cela garantit que les opérations **pixel à pixel** sont correctes et significatives.
##### **Même projection**
Les rasters peuvent être définis dans des systèmes de coordonnées différentes(ex. WGS84, UTM). Pour croiser plusieurs couches, il faut les projeter dans un **CRS commun**
**Projection choisie**: `EPSG:4326`(longitude/latitude)
> Cela permet à tous les pixels de référencer les **mêmes coordonnées géographiques**

##### **Même résolution spatiale**
Les images utilisées peuvent avoir des **résolutions différentes** (ex. 100m pour GHSL, 450m pour VIIRS). Il faut les harmoniser pour garantir un recouvrement correct.
**Résolution commune**: `100 mètres` (env. 0.0008333°)
> Cela assure que chaque pixel représente la **même surface au sol** dans tous les couches

##### **Même point d'origine(alignement spatial)**
Même avec un CRS commune et une résolution commune, deux rasters peuvent avoir des **grilles décalées** si leurs **points d'origine(coin supérieur gauche)** différent. Cela fausse les opérations de croisement.
**Point d'origine défini manuellement**

- longitude: `-13.45555`
- latitude: `15.83167`
> Cela permet de garantir que les pixels sont **parfaitement superposés**

In [8]:
# Définir une fonction pour la reprojection
def reprojection(input_image, ref_proj):

    
    image_updated = input_image.reproject(crs=ref_proj.crs(),
                                        crsTransform=ref_proj.getInfo()['transform'])  # Origine des pixels
    return image_updated


In [9]:
""" Définir une projection alignée par défaut 100 m latitude = 0.000898°, 100 m longitude = 0.000927° pour le Sénégal"""
default_proj = ee.Projection('EPSG:4326',
                               [0.000927, 0, -13.45555, 0, -0.000898, 15.83167])
img = ee.Image(0).toFloat().setDefaultProjection(default_proj)
ref_proj = img.projection()
population_updated = reprojection(population, ref_proj) # reprojection appliquée à l'image `population`
viirs_updated = reprojection(viirs, ref_proj)           # reprojection appliquée à l'image `viirs`

### Définition des seuils
#### Définition du seuil de densité de la population
Pour identifier les zones sigificativement peuplées à partir d'un raster population(ex.GHSL), il est important de fixer un **seui de densité pertinent**.

- Determiner la densité réelle de la commune étudiée *(en habitants par km2, issue de données officielles)*
- Convertir cette densité en *nombre d'habitants par pixel*, en fonction de la résolution du raster utilisée: si résolution =`100 m ` (pixel = 0.01 km2)
- Utiliser le seuil obtenu pour binariser la couche de population.
#### Définition du seuil d'éclairage nocturne
Pour identifier efficacement les zones non éclairées à partir des données VIIRS, il est essentiel de définir un **seuil de radiance** pertinent.

- Choisir deux zones contrastées au sein de la commune: une zone clairement **éclairé** (ex. centre urbain, axe principal), une zone **non éclairée** (ex. quartier périphérique, village, zone rurale).
- Afficher l'image VIIRS (ex. `avg_rad`) sur la carte.
- Jouer progressivement sur la valeur du seuil jusqu'à ce que la visualisation du masque reflète fidèlement la répartition réelle de la lumière observée.
- Une fois satisfait, retenir ce seuil comme seuil de classification binaire.

In [10]:
# Définir deux points pour le calcul de seuil d'éclairage nocturne
light_point = ee.Geometry.Point([-13.45555, 15.83167])
black_point = ee.Geometry.Point([-13.47555, 15.83972])
""" En ajustant la valeur à partir de ces deux point, on obtient un seuil d'éclairage nocturne égal à 0.45 """

" En ajustant la valeur à partir de ces deux point, on obtient un seuil d'éclairage nocturne égal à 0.45 "

### Statistiques exploitables sur le raster 
Il s'agit de calculer les statistiques d'habitation, et de vulnérablité à l'obscurité pour la commune de Bokidiawé

In [11]:
def vulnerabl_areas_statistics_calculation(pop_image, viirs_image, commune_geom, commune_name, pop_threshold=1.24, 
                                            viirs_threshold=0.45, scale=100):
    pixel_area = ee.Image.pixelArea()
    # Masques
    binary_population = pop_image.select('population_count').gte(pop_threshold)
    binary_viirs = viirs_image.select('avg_rad').gte(viirs_threshold)
   
    inhabited_unlit = binary_population.And(binary_viirs.Not())

    parameters = {
        'reducer': ee.Reducer.sum(),
        'geometry': commune_geom,
        'scale': scale,
        'maxPixels': 1e13
    }
    # Surface totale
    total_area = pixel_area.reduceRegion(**parameters).get('area')

    # Surface habitée
    living_area = pixel_area.updateMask(binary_population).reduceRegion(**parameters).get('area')

    
    # Surface habitée non éclairée
    vulnerable_area = pixel_area.updateMask(inhabited_unlit).reduceRegion(**parameters).get('area')

    # Population vulnerable
    unlit_resident = population.updateMask(inhabited_unlit).reduceRegion(**parameters).get('population_count')

    # Convertir les surface en hectares
    stats = ee.Dictionary({
        'Commune': commune_name,
        'Superficie totale (ha)': ee.Number(total_area).divide(10000),
        'Superficie habitéé (ha)': ee.Number(living_area).divide(10000),
        'Surface vulnérable (ha)': ee.Number(vulnerable_area).divide(10000),
        'Population vulnerable': unlit_resident
    })

    return stats, binary_population, binary_viirs, inhabited_unlit


In [12]:
stats, binary_population, binary_viirs, inhabited_unlit = vulnerabl_areas_statistics_calculation(pop_image=population_updated,
                                                                                                     viirs_image=viirs_updated, 
                                                                                                     commune_geom=ee_geom, 
                                                                                                     commune_name='BOKIDIAWE', 
                                                                                                     pop_threshold=1.24, 
                                                                                                     viirs_threshold=0.45, 
                                                                                                     scale=100)
stats.getInfo()

{'Commune': 'BOKIDIAWE',
 'Population vulnerable': 6456.888269709985,
 'Superficie habitéé (ha)': 2326.7290133237593,
 'Superficie totale (ha)': 39744.99574462664,
 'Surface vulnérable (ha)': 603.4887038897824}

#### Interprétation
Environ **26 %** de la surface habitée de la commune de Bokidiawé n'est pas éclairée la nuit, exposant plus de **6000 personnes** à des conditions de vulnérabilité énergétique.

#### Visualisation des zones habitées non éclairées

In [15]:
m = geemap.Map()

m.addLayer(binary_population.updateMask(binary_population), {'min':0, 'max':1, 'palette':['green']}, 'Zones habitées')
m.addLayer(binary_viirs, {'min':0, 'max':1, 'palette': ['black', 'yellow']}, 'Zones éclairées')
m.addLayer(inhabited_unlit.updateMask(inhabited_unlit), {'min':0, 'max':1, 'palette':['red']}, 'Habités non éclairés')
m.addLayer(light_point, {'color': 'white'}, 'Point éclairé')
m.addLayer(black_point, {'color': 'grey'}, 'Point sombre')
m.centerObject(ee_geom, 10)
m.save('carte_zones_habitees_non_eclairees.html')
m

Map(center=[15.924526321686766, -13.453697042991458], controls=(WidgetControl(options=['position', 'transparen…